# Workflow for kinetic parameter estimation

This notebook contains a workflow for kinetic parameter estimation based on photometric raw data.
In the next cell, `MTPHandler` and `EnzymePynetics` are installed and imported.

In [1]:
import sys

try:
    from MTPHandler.core.plate import Plate
    from MTPHandler.core.reactant import Reactant
    from MTPHandler.core.protein import Protein
    from MTPHandler.ioutils import parse_magellan

except ModuleNotFoundError:
    !{sys.executable} - m pip install git+https: // github.com/FAIRChemistry/MTPHandler.git@ci - -quiet
    !git clone https: // github.com/haeussma/laccase_kinetics.git
    from MTPHandler.core.plate import Plate
    from MTPHandler.core.reactant import Reactant
    from MTPHandler.core.protein import Protein
    from MTPHandler.ioutils import parse_magellan


By executing the next cell the installed packages are imported. Occasionally, Colab has an issue with the import. In this case got to `Runtime` and click `Restart runtime`. Then execute the cell again.

## Load experimental data

In the following cell, the Magellan output file is read in and the data is mapped to the `Plate` object. 

In [2]:
# Path to spectrometer output file
path = "laccase_kinetics/laccase_kinetics/Laccase_kinetic_woTween_Rawdata.xlsx"

# Load plate
plate = Plate.from_reader(
    reader=parse_magellan,
    path=path,
    ph=3,
    wavelength=420,
)


In [3]:
# Visualize plate data
plate.visualize()

### Define reactants

Next up, all involved reactants and proteins present in any of the MTP wells are defined.

In [4]:
buffer = plate.add_reactant(id="s0", name="Buffer", constant=True)

abts = plate.add_reactant(id="s1", name="ABTS", constant=False)

abts_radical = plate.add_reactant(id="s2", name="ABTS radical", constant=False)

laccase = plate.add_protein(
    id="p0", name="Laccase", constant=True, sequence="MSSKSKPKDVKV"
)


### Assign initial conditions

Species can be assigned to all wells, row-wise, or column-wise one species at a time. Thereby, multiple rows with identical initial concentrations can be assigned simultaneously.  
Please not that currently all units need to be of the same magnitude. Otherwise the later modeling results will not be correct.

In [5]:
# Assign buffer concentration
plate.assign_species(to="all", species=buffer,
                     init_conc=100, conc_unit="mmol / l")

# Assign substrate concentrations
plate.assign_species(
    to="rows",
    species=abts,
    ids=["A", "B", "C", "D"],
    init_conc=[0, 0.3, 0.6, 0.9, 5, 30, 50, 70],
    conc_unit="mmol / l",
)

plate.assign_species(
    to="rows",
    species=abts,
    ids=["E", "F", "G", "H"],
    init_conc=[0.15, 0.45, 0.75, 1, 10, 40, 60, 80],
    conc_unit="mmol / l",
)

plate.assign_species(to="all", species=abts_radical,
                     init_conc=0, conc_unit="mmol / l")


# Assign enzyme concentrations
plate.assign_species(to="all", species=laccase,
                     init_conc=0.006, conc_unit="mmol / l")

Assigned Buffer to all wells.
Assigned ABTS with concentrations of [0, 0.3, 0.6, 0.9, 5, 30, 50, 70] mmol / l to row A.
Assigned ABTS with concentrations of [0, 0.3, 0.6, 0.9, 5, 30, 50, 70] mmol / l to row B.
Assigned ABTS with concentrations of [0, 0.3, 0.6, 0.9, 5, 30, 50, 70] mmol / l to row C.
Assigned ABTS with concentrations of [0, 0.3, 0.6, 0.9, 5, 30, 50, 70] mmol / l to row D.
Assigned ABTS with concentrations of [0.15, 0.45, 0.75, 1, 10, 40, 60, 80] mmol / l to row E.
Assigned ABTS with concentrations of [0.15, 0.45, 0.75, 1, 10, 40, 60, 80] mmol / l to row F.
Assigned ABTS with concentrations of [0.15, 0.45, 0.75, 1, 10, 40, 60, 80] mmol / l to row G.
Assigned ABTS with concentrations of [0.15, 0.45, 0.75, 1, 10, 40, 60, 80] mmol / l to row H.
Assigned ABTS radical to all wells.
Assigned Laccase to all wells.


### Create EnzymeML document

As the last part of data treatment, the `Plate` object is converted to an `EnzymeMLDocument` object. This object is the basis for all further processing steps.

In [6]:
enzymeml = plate.to_enzymeml(
    name="laccase kinetic assay",
    detected_reactant=abts_radical,
    wavelength=420,
    path="lac_kinetics.json",
    ignore_blank_status=True,
)


Found 64 catalyzed wells


## Concentration calculation

In this case I assume that the concentration should be calculated via the extinction coefficient for ABTS radical. Therefore, I've prepared quick methods for applying an extinction coefficient to the previously created `EnzymeMLDocument` object. The methods are defined in the next cell.

In [7]:
def calculate_concentration(absorptions, ext_coeff: float, pathlength: float):
    """Calculated concentrations for a list of absorption values,
    given an extinction coefficient and path length.
    """

    return [absorption / (ext_coeff * pathlength) for absorption in absorptions]


def apply_extinction_coef(enzymeml, ext_coeff: float, pathlength: float):
    """Applies extinction coefficient to all replicates in an EnzymeML document."""

    for measurement in enzymeml.measurements:
        for species in measurement.species:
            if not species.replicates:
                continue
            for replicate in species.replicates:
                replicate.data = calculate_concentration(
                    replicate.data, ext_coeff, pathlength
                )

I assumed an extinction coefficient of 36 $mM^{-1} cm^{-1}$ and path length of 0.4 $cm$ for ABTS radical. This value is taken from the literature. The extinction coefficient is applied to all wells containing ABTS radical. The resulting concentrations are stored in the `EnzymeMLDocument` object.

In [8]:
# Apply extinction coefficient to EnzymeML document
apply_extinction_coef(enzymeml=enzymeml, ext_coeff=36, pathlength=0.4)


## Parameter estimation

In [18]:
try:
    from EnzymePynetics.core.estimator import Estimator

except ModuleNotFoundError:
    !{sys.executable} - m pip install git+https: // github.com/haeussma/EnzymePynetics.git@enzymeml-platform - -quiet
    from EnzymePynetics.core.estimator import Estimator


In [10]:
# Initialize estimator
estimator, _ = Estimator.from_enzymeml(enzymeml, abts_radical)


In [11]:
# Define reaction
oxidation = estimator.add_reaction(
    id="r1", name="Oxidation", educt=abts, product=abts_radical, enzyme=laccase
)


In the next cell, possible substrate rate laws are defined. Each rate law is fitted to the measurement data individually. 

In [12]:
# Substrate rate laws
michaelis = estimator.add_model(
    id="model1",
    name="michaelis-menten",
    equation="substrate = -substrate * enzyme * k_cat / (K_m + substrate)",
)

competitive_product = estimator.add_model(
    id="model3",
    name="competitive product inhibition",
    equation="substrate = -substrate * enzyme * k_cat / (K_m * (1 + product / K_ic) + substrate)",
)

substrate_inhibition = estimator.add_model(
    id="model4",
    name="substrate inhibition",
    equation="substrate = -k_cat * enzyme * substrate / (K_m + ((1+(substrate/K_iu))*substrate))",
)


In [13]:
# Fit models
estimator.fit_models()


Fitting michaelis-menten


/Users/max/Documents/GitHub/software-driven-rdm/sdRDM/base/datamodel.py:326: UserWarning:

No 'URL' and 'Commit' specified. This model might not be re-usable.



Fitting competitive product inhibition
Fitting substrate inhibition


,AIC,k_cat,K_m,K_ic,K_iu,k_ie
,,1 / sec,mmol / l,mmol / l,mmol / l,1 / sec
Model,,,,,,
substrate inhibition,-13539,0.02,0.27,,3998.11,
michaelis-menten,-13538,0.02,0.26,,,
competitive product inhibition,-13536,0.02,0.26,41.80,,


In [14]:
# Visualize models
estimator.visualize()


## Save modeling results to EnzymeML

In [15]:
# Get the reaction system
reaction_system = estimator.get_reaction_system("michaelis-menten")

# Save the reaction system to EnzymeML
result_doc = estimator.to_enzymeml(enzymeml=enzymeml, reaction_system=reaction_system)

In [16]:
print(enzymeml)


EnzymeMLDocument
├── id = enzymemldocument0
├── name = laccase kinetic assay
├── created = 2023-10-30 23:31:56.465737
├── vessels
│   └── 0
│       └── Vessel
│           ├── id = plate0
│           ├── name = MTP 96 well
│           ├── volume = 200.0
│           ├── unit = ul
│           └── constant = True
├── proteins
│   └── 0
│       └── Protein
│           ├── id = p0
│           ├── name = Laccase
│           ├── vessel_id = plate0
│           ├── constant = True
│           ├── sequence = MSSKSKPKDVKV
│           └── ontology = SBO:0000013
├── reactants
│   ├── 0
│   │   └── Reactant
│   │       ├── id = s0
│   │       ├── name = Buffer
│   │       ├── vessel_id = plate0
│   │       ├── constant = True
│   │       └── ontology = SBO:0000247
│   ├── 1
│   │   └── Reactant
│   │       ├── id = s1
│   │       ├── name = ABTS
│   │       ├── vessel_id = plate0
│   │       ├── constant = False
│   │       └── ontology = SBO:0000247
│   └── 2
│       └── Reactant
│           ├── id 